# Tema 7

Sa se creeze si antreneze o retea de tip convolutional pentru unul din seturile de date [CIFAR100](https://www.cs.toronto.edu/~kriz/cifar.html), [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html), [Food101](https://pytorch.org/vision/main/generated/torchvision.datasets.Food101.html). Intrarea este RGB, de forma 32 x 32 x 3 (la CIFAR10, 100) si dimensiune (max 512) x (max 512) x 3 pentru Food101. 

Cerinte:
1. Efectuati minim 50 de epoci de instruire. 
1. Reprezentati grafic evolutia functiei de cost si a acuratetei, masurate pe setul de antrenare si de validare, folosind tensorboard. **Fisierele tfevents vor fi incluse in arhiva zip care se preda pe elearning**. 
1. Acuratetea unui model neantrenat este de in medie 1/(numarul de clase), adica modelul decide aleator o clasa pentru fiecare input. Modelul vostru va trebui sa ajunga mult peste acest prag de baza. 
1. Cautare de learning rate (LR) cat mai bun: odata gasit arhitectura care performeaza peste pragul de baza, incercati 3 valori diferite pentru hiperparametrul learning rate. Modelul va fi antrenat pe un prcent de 90% de imagini alese aleator din setul de antrenare si validat pe restul de 10% imagini din setul de antrenare. Acel LR care produce cea mai buna acuratete pe setul de validare va fi folosit pentru antrenarea peste toate imaginile din setul initial de antrenare si in final testat pe setul de testare. Setul de testare nu se va folosi in etapa de antrenare sau de validare (cautare de LR optimal).
2. Se va face scalarea datelor din setul de antrenare si de validare, calculand media si deviatia standard pe setul de antrenare; vor rezulta 2 tensori cu cate 3 dimensiuni (cate o valoare pentru canalele R, G, B); se poate folosi functia `compute_mean_std` din prezentarea de laborator. 

Tema va fi predata pana in **22 mai 2024 ora 23** pe elearning. Notarea ei se va face doar in prezenta studentului.

## Import de pachete

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from typing import Tuple

import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

In [ ]:
# !pip install torchinfo
from torchinfo import summary

# !pip install tensorboard
from torch.utils.tensorboard import SummaryWriter

## Pregatire dataset, dataloader

Codul de mai jos e un punct de plecare posibil / exemplu pentru setul CIFAR100. Utilizarea acestui cod nu e mandatorie. 

In [ ]:
def compute_mean_std(train_data: datasets.MNIST) -> Tuple[torch.Tensor, torch.Tensor]:
    channel_sum, channel_squared_sum = 0, 0
    for data, _ in train_data:
        channel_sum += torch.mean(data, dim=[1, 2])
        channel_squared_sum += torch.mean(data**2, dim=[1, 2])

    mean = channel_sum / len(train_data)
    std = (channel_squared_sum / len(train_data) - mean**2) ** 0.5

    return mean, std

In [ ]:
# se poate ajusta dimensiunea batchului
batch_size = 64

train_dataset = datasets.CIFAR100('./data', 
                               train=True, 
                               download=True, 
                               transform=transforms.ToTensor())

assert len(train_dataset) == 50000

# pentru standardizarea datelor calculam valoare medie si deviatia standard pe cele 3 canale de intrare ale fiecare imagini
mean_train, std_train = compute_mean_std(train_dataset)

# transformarile care se aplica pentru fiecare imagine din set de train sau de validare
image_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_train, std_train)])

train_dataset = datasets.CIFAR100('./data', 
                               train=True, 
                               download=True, 
                               transform=image_transform)

test_dataset = datasets.CIFAR100('./data', 
                                    train=False, 
                                    transform=image_transform)

assert len(test_dataset) == 10000



In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False)

Nota: daca celula de plot produce eroare, atunci se foloseste workaround de la [https://stackoverflow.com/questions/53014306/error-15-initializing-libiomp5-dylib-but-found-libiomp5-dylib-already-initial](https://stackoverflow.com/questions/53014306/error-15-initializing-libiomp5-dylib-but-found-libiomp5-dylib-already-initial):"

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

In [ ]:
pltsize=1
plt.figure(figsize=(8, 6))

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.axis('off')
    plt.imshow(X_train[i,:,:,:].numpy().reshape(3,32,32).transpose(1,2,0))
    plt.title('Class: '+str(y_train[i].item()))

## Functii: antrenare, validare, desenare

Definiti in mod convenabil functii de antrenare si de validare. Puteti reutiliza cod din notebook-urile anterioare. 

In [ ]:
def train(model: nn.Module,
          optimizer: torch.optim.Optimizer,
          criterion: callable,
          train_loader: torch.utils.data.DataLoader,
          epochs: int,
          device: torch.device,
          log_interval: int = 100) -> None:
    # seteaza modelul in modul de antrenare
    model.train()
    
    # ciclul de invatare pentru fiecare batch din setul de antrenare
    for batch_idx, (data, target) in enumerate(train_loader):
        # copiere date pe device-ul de lucru
        data, target = data.to(device), target.to(device)
        
        # setarea gradientilor pe zero, deoarece acestia se acumuleaza, nu se inlocuiesc
        model.zero_grad()
        
        # inferenta modelului, adica calculul iesirilor prin propagarea datelor de intrare in retea
        output = model(data)
        
        # calculul valorii functiei de cost
        loss = criterion(output, target)
        
        # calculul gradientilor
        loss.backward()
        
        # actualizarea ponderilor
        optimizer.step()
        
        # afisarea progresului in timpul antrenarii
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: [ {epochs}] [{batch_idx * len(data)} / {len(train_loader.dataset)} ] Loss: {loss.item()}\
                  Accuracy: {100. * batch_idx / len(train_loader):.2f}%')
            


In [ ]:
@torch.no_grad()
def test(model: nn.Module,
         criterion: callable,
         test_loader: torch.utils.data.DataLoader,
         device: torch.device,
         epoch: int,
         writer: SummaryWriter,
         subset: str) -> None:
    # setarea modelului in modul de evaluare
    model.eval()
    
    test_loss, correct = 0, 0
    
    # ciclul de evaluare pentru fiecare batch din setul de validare
    for data, target in test_loader:
        # copierea datelor pe device-ul de lucru
        data, target = data.to(device), target.to(device)
        
        # calculul iesirilor modelului, adica inferenta
        output: torch.Tensor = model(data)
        
        # calculul valorii functiei de cost pentru batch-ul curent
        test_loss += criterion(output, target).item()
        
        # determinarea clasei prezise de model
        pred : torch.Tensor = output.argmax(dim=1, keepdim=True)
        
        # determinarea numarului de clasificari corecte
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    # calculul valorii medii a functiei de cost si acuratetii pe setul de validare
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    with writer:
        # scrierea valorilor in fisierul de log-uri
        writer.add_scalar(f'Loss/{subset}', test_loss, global_step=epoch)
        writer.add_scalar(f'Accuracy/{subset}', accuracy, global_step=epoch)
        
    print(f'Epoch {epoch}: {subset} set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')

## Definire model

In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0, bias=True)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True)
        
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0, bias=True)
        
        self.fc1 = nn.Linear(in_features=120, out_features=84, bias=True)
        self.fc2 = nn.Linear(in_features=84, out_features=100, bias=True)
        
        self.activation = nn.ReLU(inplace=True)
        
        
    def forward(self, x):
       x = self.conv1(x)
       
       x = self.activation(x)
       
       x = self.pool1(x)
       
       x = self.conv2(x)
       
       x = self.activation(x)
       
       x = self.pool2(x)
       
       x = self.conv3(x)
       
       x = self.activation(x)
       
       x = x.view(x.size(0), -1)
       
       x = self.fc1(x)
       x = self.activation(x)
       x = self.fc2(x)
       
       return x

In [ ]:
from time import strftime

train90 = int(0.9 * len(train_dataset))
train_dataset, valid_dataset = torch.utils.data.random_split(train_loader.dataset, [train90, len(train_loader.dataset) - train90])
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)

lenet5 = LeNet5().to(device)

time_str = strftime("%Y_%m_%d_%H_%M_%S")
writer = SummaryWriter(f'runs/lenet5-cifar100_1.0{time_str}')
writer.add_graph(lenet5, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.Adam(lenet5.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

summary(lenet5, input_size=(batch_size, 3, 32, 32))

## Antrenare si validare

Se vor tipari si prezenta ploturi (loss, acuratete) pe seturile de antrenare si validare la finalul fiecarei epoci.

In [172]:
epochs = 50

def antrenare_validare(lenet5, criterion, optimizer, train_loader, valid_loader, epochs, device, writer):
    print('Start training...')
    test(lenet5, criterion, train_loader, device, 0, writer, 'Train')
    print('-' * 50)
    test(lenet5, criterion, valid_loader, device, 0, writer, 'Validation')
    print('\n')
    for epoch in range(1, epochs + 1):
        train(lenet5, optimizer, criterion, train_loader, epoch, device)
        test(lenet5, criterion, train_loader, device, epoch, writer, 'Train')
        print('\n')
        test(lenet5, criterion, valid_loader, device, epoch, writer, 'Validation')
        print('-' * 50)

antrenare_validare(lenet5, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Epoch 0: Train set: Average loss: 0.0721, Accuracy: 531/45000 (1.18%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0728, Accuracy: 54/5000 (1.08%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.5961456298828125                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.222434997558594                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.094571113586426                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 3.8646650314331055                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 3.8996126651763916                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 3.672469139099121                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 3.502354145050049                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 3.682504177093506                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0558, Accuracy: 6681/45000 

In [173]:
# resetam ponderile modelului
lenet5_1 = LeNet5().to(device)

writer = SummaryWriter(f'runs/lenet5-cifar100_1.1{time_str}')
writer.add_graph(lenet5_1, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.Adam(lenet5_1.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

summary(lenet5_1, input_size=(batch_size, 3, 32, 32))

antrenare_validare(lenet5_1, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Start training...
Epoch 0: Train set: Average loss: 0.0721, Accuracy: 384/45000 (0.85%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0728, Accuracy: 35/5000 (0.70%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.610024929046631                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.480613708496094                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.192436695098877                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 3.977532386779785                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.166882038116455                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 4.0477776527404785                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 3.63189959526062                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 3.741849899291992                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0583, Accura

In [187]:
# resetam ponderile modelului
lenet5_2 = LeNet5().to(device)

writer = SummaryWriter(f'runs/lenet5-cifar100_1.2{time_str}')
writer.add_graph(lenet5_2, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.Adam(lenet5_2.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

summary(lenet5_2, input_size=(batch_size, 3, 32, 32))


antrenare_validare(lenet5_2, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Start training...
Epoch 0: Train set: Average loss: 0.0721, Accuracy: 444/45000 (0.99%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0728, Accuracy: 56/5000 (1.12%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.591494083404541                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.340390682220459                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.122034549713135                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 3.9043638706207275                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.1803460121154785                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 3.989807367324829                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 3.823094367980957                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 4.18647575378418                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0595, Accur

KeyboardInterrupt: 

In [174]:
# resetam ponderile modelului
lenet5_3 = LeNet5().to(device)

writer = SummaryWriter(f'runs/lenet5-cifar100_1.3{time_str}')
writer.add_graph(lenet5_3, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.Adam(lenet5_3.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

summary(lenet5_3, input_size=(batch_size, 3, 32, 32))


antrenare_validare(lenet5_3, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Start training...
Epoch 0: Train set: Average loss: 0.0721, Accuracy: 452/45000 (1.00%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0728, Accuracy: 48/5000 (0.96%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.599170207977295                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.612636089324951                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.595944404602051                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 4.610259532928467                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.591670989990234                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 4.628706932067871                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 4.614070415496826                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 4.612863540649414                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0721, Accura

In [175]:
# resetam ponderile modelului
lenet5_4 = LeNet5().to(device)

writer = SummaryWriter(f'runs/lenet5-cifar100_2.0{time_str}')
writer.add_graph(lenet5_4, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.SGD(lenet5_4.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

summary(lenet5_4, input_size=(batch_size, 3, 32, 32))


antrenare_validare(lenet5_4, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Start training...
Epoch 0: Train set: Average loss: 0.0721, Accuracy: 463/45000 (1.03%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0729, Accuracy: 56/5000 (1.12%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.595675468444824                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.614591598510742                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.611462593078613                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 4.615906715393066                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.591394424438477                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 4.607371807098389                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 4.6056084632873535                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 4.598531723022461                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0720, Accur

In [176]:
# resetam ponderile modelului
lenet5_5 = LeNet5().to(device)

writer = SummaryWriter(f'runs/lenet5-cifar100_2.1{time_str}')
writer.add_graph(lenet5_5, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.SGD(lenet5_5.parameters(), lr=0.0025, momentum=0.9)
criterion = nn.CrossEntropyLoss()

summary(lenet5_5, input_size=(batch_size, 3, 32, 32))

antrenare_validare(lenet5_5, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Start training...
Epoch 0: Train set: Average loss: 0.0721, Accuracy: 342/45000 (0.76%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0728, Accuracy: 34/5000 (0.68%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.6146697998046875                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.59616231918335                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.612462997436523                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 4.60885763168335                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.589854717254639                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 4.617699146270752                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 4.569274425506592                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 4.490810394287109                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0706, Accurac

In [177]:
# resetam ponderile modelului
lenet5_6 = LeNet5().to(device)

writer = SummaryWriter(f'runs/lenet5-cifar100_2.2{time_str}')
writer.add_graph(lenet5_6, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.SGD(lenet5_6.parameters(), lr=0.005, momentum=0.9)
criterion = nn.CrossEntropyLoss()

summary(lenet5_6, input_size=(batch_size, 3, 32, 32))

antrenare_validare(lenet5_6, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Start training...
Epoch 0: Train set: Average loss: 0.0721, Accuracy: 482/45000 (1.07%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0728, Accuracy: 54/5000 (1.08%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.59681510925293                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.6093268394470215                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.595822811126709                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 4.558289051055908                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.487477779388428                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 4.463357448577881                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 4.450806140899658                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 4.440591335296631                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0663, Accura

In [178]:
# resetam ponderile modelului
lenet5_7 = LeNet5().to(device)

writer = SummaryWriter(f'runs/lenet5-cifar100_2.3{time_str}')
writer.add_graph(lenet5_7, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.SGD(lenet5_7.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

summary(lenet5_7, input_size=(batch_size, 3, 32, 32))

antrenare_validare(lenet5_7, criterion, optimizer, train_loader, valid_loader, epochs, device, writer)

Start training...
Epoch 0: Train set: Average loss: 0.0721, Accuracy: 449/45000 (1.00%)
--------------------------------------------------
Epoch 0: Validation set: Average loss: 0.0728, Accuracy: 50/5000 (1.00%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.604579448699951                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.597751617431641                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.560432434082031                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 4.501109600067139                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.290459632873535                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 4.120670318603516                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 4.231616020202637                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 3.8103394508361816                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0610, Accur

In [191]:
# cea mai buna retea este lenet5_4
# testam reteaua pe setul de testare

lenet_final = LeNet5().to(device)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

writer = SummaryWriter(f'runs/lenet5-cifar100_final{time_str}')
writer.add_graph(lenet_final, next(iter(train_loader))[0].to(device))

optimizer = torch.optim.SGD(lenet_final.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()


test(lenet_final, criterion, train_loader, device, 0, writer, 'Train')
print('-' * 50)
test(lenet_final, criterion, test_loader, device, 0, writer, 'Test')
print('\n')
for epoch in range(1, epochs + 1):
    train(lenet_final, optimizer, criterion, train_loader, epoch, device)
    test(lenet_final, criterion, train_loader, device, epoch, writer, 'Train')
    print('\n')
    test(lenet_final, criterion, test_loader, device, epoch, writer, 'Test')
    print('-' * 50)

Epoch 0: Train set: Average loss: 0.0721, Accuracy: 416/45000 (0.92%)
--------------------------------------------------
Epoch 0: Test set: Average loss: 0.0723, Accuracy: 107/10000 (1.07%)


Train Epoch: [ 1] [0 / 45000 ] Loss: 4.621586322784424                  Accuracy: 0.00%
Train Epoch: [ 1] [6400 / 45000 ] Loss: 4.608044147491455                  Accuracy: 14.20%
Train Epoch: [ 1] [12800 / 45000 ] Loss: 4.615594387054443                  Accuracy: 28.41%
Train Epoch: [ 1] [19200 / 45000 ] Loss: 4.621655464172363                  Accuracy: 42.61%
Train Epoch: [ 1] [25600 / 45000 ] Loss: 4.593959808349609                  Accuracy: 56.82%
Train Epoch: [ 1] [32000 / 45000 ] Loss: 4.607038497924805                  Accuracy: 71.02%
Train Epoch: [ 1] [38400 / 45000 ] Loss: 4.611080169677734                  Accuracy: 85.23%
Train Epoch: [ 1] [44800 / 45000 ] Loss: 4.6155242919921875                  Accuracy: 99.43%
Epoch 1: Train set: Average loss: 0.0720, Accuracy: 644/45000 (1.43%)